In [7]:
import pandas as pd
from random import randint

In [8]:
#file_name = '../EIL51.xlsx'
file_name = '../Lab Data.xlsx'
distances = pd.read_excel(file_name, index_col=None)

TOUR_LENGTH = len(distances)

### Greedy Randomized Heuristic

In [121]:
def valid(current_tour):
    # Drop cities that are already in the current tour
    valid_nbrs = [element for element in range(TOUR_LENGTH) if element not in current_tour]
    return valid_nbrs


def nearestish_neighbor(starting_city=None, tour=None, top_l=3):   
    '''
        Optionally takes in a starting city, tour, and/or alpha value.
        If no starting city is provided a random one is chosen
        If no current tour is provided it is instantiated as an empty list
        The top_l is the number of candidates passed to the semi-greedy algorithm

        Returns a tour and the corresponding length
    '''
    tour = tour or []
    current_city = starting_city if isinstance(starting_city, int) else randint(0,TOUR_LENGTH)
    tour_length = 0
    tour.append(current_city)   # add starting city to tour    
    
    for _ in range(TOUR_LENGTH-1):
        next_city, distance_to_next = nearestish(current_city, tour, top_l)
        tour.append(next_city)
        current_city = next_city
        tour_length += distance_to_next

    tour.append(tour[0]) # The TSP requires that the salesman goes back home at the end.
    tour_length += distances[tour[-2]][tour[-1]] # Add the return to origin distance
    
    return tour, tour_length


def nearestish(current_city, current_tour, top_l=3):
    '''
        Given current city and list of current tour find a nearish neighbor
        from within the l shortest options.

        Returns index of next city and distance to that city
    '''
    # Get distances from current city
    # Filter for valid destinations
    candidates = distances[current_city].filter(valid(current_tour))

    # Select l best options (RCL)
    rcl = candidates.nsmallest(top_l)

    # Randomly Select from RCL
    nearish_city = rcl.sample(1)

    # Return Destination and distance to it
    return nearish_city.index[0], nearish_city.values[0]

### Local Search

In [122]:
nearestish_neighbor()

2    4
3    5
7    6
Name: 6, dtype: int64
0     15
12    23
5     40
Name: 2, dtype: int64
0    23
8    28
4    49
Name: 12, dtype: int64
4    12
1    13
9    27
Name: 8, dtype: int64
10    7
13    8
14    9
Name: 9, dtype: int64
1     11
11    34
4     47
Name: 13, dtype: int64
4     3
3    32
7    36
Name: 11, dtype: int64
0    26
1    40
7    45
Name: 4, dtype: int64
14     1
3     20
5     59
Name: 1, dtype: int64
0      4
3     24
10    26
Name: 14, dtype: int64
7     2
3    65
0    90
Name: 10, dtype: int64
5    73
7    80
0    99
Name: 3, dtype: int64
7    81
0    93
Name: 5, dtype: int64
7    20
Name: 0, dtype: int64
[6, 2, 12, 8, 9, 13, 11, 4, 1, 14, 10, 3, 5, 0, 7]
445


([6, 2, 12, 8, 9, 13, 11, 4, 1, 14, 10, 3, 5, 0, 7, 6], 451)

In [120]:
a.min()

0

In [102]:
a = distances[3]#.filter([4,5,6])
print(a)
b = a[a.gt(70)]
print(b)
c = b.sample(1)
print(c)
i = c.index[0]
print(i)
v = c.values[0]
print(v)
l = []
l.append(i)
l.append(1)
print(l)

0     99
1     20
2     76
3      0
4     75
5     73
6      5
7     80
8     57
9     12
10    65
11    32
12    94
13    76
14    24
Name: 3, dtype: int64
0     99
2     76
4     75
5     73
7     80
12    94
13    76
Name: 3, dtype: int64
13    76
Name: 3, dtype: int64
13
76
[13, 1]


In [ ]:
tour = []
tour_length = 0

NN(1,tour)  #This finds the nearest neighbor tour starting at city 1.

for i in range(len(tsp_data)):
    tour = []
    tour_length = 0
    tour, tour_length = NN(i, tour)
    print(tour, tour_length)